In [1]:
from pprint import pprint
from time import sleep
import requests
from bs4 import BeautifulSoup

# url_list = [
#     'https://www.subito.it/annunci-italia/vendita/usato/?q=spectrum+sinclair&o=1',
#     'https://www.subito.it/annunci-italia/vendita/usato/?q=spectrum+sinclair&o=2'
# ]

base_url = 'https://www.subito.it/annunci-italia/vendita/usato/'


def scrape_page(page):
    query_str = {
        'q': 'spectrum+sinclair',
        'o': page
    }

    result = requests.get(base_url, params=query_str)

    if result.status_code == 200:
        soup = BeautifulSoup(result.content, 'html.parser')

        # Controllare se non è una pagina vuota
        no_results = soup.select('div[class*=ErrorLayout_container__]')
        if no_results:
            raise requests.RequestException('Non ci sono più pagine da analizzare.')

        products = soup.select('div.items__item.item-card')
        # pprint(products)

        items = []

        for adv in products:
            data = {
                'name': adv.select('h2[class*=ItemTitle-module_item-title__]'),
                # 'name': adv.select('h2[class*=ItemTitle-module_item-title__]')[0] \
                #     .get_text() if adv.select('h2[class*=ItemTitle-module_item-title__]') else None,
                'price': adv.select('p.price'),
                'town': adv.select('span[class*=index-module_town__]'),
                'city': adv.select('span.city'),
                'shipping': adv.select('span.shipping-badge'),
                'sold': adv.select('span.item-sold-badge'),  
            }

            for key, value in data.items():
                if value:
                    data[key] = value[0].get_text()
                else:
                    data[key] = None

                # data[key] = value[0].get_text() if value else None

            data.update({
                'price': float(data['price']
                        .replace('Spedizione disponibile', '')
                        .replace('\xa0€', '')
                        .replace('Venduto', '')) if data['price'] else None,
                'url': adv.select('a.link')[0]['href'],
            })

            # [0].get_text().replace('Spedizione disponibile', '').replace('Venduto', ''),
            
            items.append(data)
        
    return items


curr_page = 1
all_data = []
# max_page = None

while True:
    try:
        items = scrape_page(curr_page)
    except requests.RequestException as err:
        print(err)
        break
    
    all_data += items
    curr_page += 1
    sleep(1)

print(all_data)

{'name': 'Sinclair ZX Spectrum 16kb',
 'price': '550\xa0€Spedizione disponibile',
 'town': 'Torino '}
{'name': 'Disco 2tb Batocera v36',
 'price': '100\xa0€Spedizione disponibile',
 'town': 'Pagani '}
{'name': 'Disco 2tb Batocera',
 'price': '100\xa0€Spedizione disponibile',
 'town': 'Pagani '}
{'name': 'Sinclair ZX Spectrum 48k',
 'price': '280\xa0€',
 'town': 'Villafranca Padovana '}
{'name': 'Computer sinclair zx spectrum 48k - testato 100%',
 'price': '150\xa0€Spedizione disponibile',
 'town': 'Salerno '}
{'name': 'Zx spectrum plus Sinclair',
 'price': '200\xa0€Spedizione disponibile',
 'town': 'Roma '}
{'name': 'Zx Spectrum 48 K Sinclair',
 'price': '99\xa0€Spedizione disponibile',
 'town': 'Milano '}
{'name': 'SINCLAIR ZX SPECTRUM 48 K originale con scatola',
 'price': '220\xa0€Spedizione disponibile',
 'town': 'Roma '}
{'name': 'Giochi per Sinclair ZX Spectrum',
 'price': '100\xa0€Spedizione disponibile',
 'town': 'Verona '}
{'name': 'Libri / Manuali Retrocomputer Spectrum Commo